In [1]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
stats = pd.read_csv("player_mvp_stats.csv")
del stats["Unnamed: 0"]
stats_all_players = stats.fillna(0)
#mvp_players = stats_all_players[(stats_all_players["FG%"] > 0.384) & (stats_all_players["MP"] > 25.6) & (stats_all_players["FT%"] > 0.422) & (stats_all_players["PTS"] > 4.7) & (stats_all_players["VORP"] > 1.1)& (stats_all_players["W/L%"] > 0.22)]
mvp_players = stats_all_players

In [3]:
stats_2024 = pd.read_csv("2024_stats.csv")
del stats_2024["Unnamed: 0"]

In [4]:
stats_2024 = stats_2024.fillna(0)
#stats_2024 = stats_2024[(stats_2024["FG%"] > 0.384) & (stats_2024["MP"] > 25.6) & (stats_2024["FT%"] > 0.422) & (stats_2024["PTS"] > 4.7) & (stats_2024["VORP"] > 1.1)& (stats_2024["W/L%"] > 0.22)]
condition = (stats_2024['W'] + stats_2024['L']) - stats_2024['G'] < 17
stats_2024 = stats_2024[condition]

In [5]:
# pd.set_option('display.max_rows', None)
# len(stats_2024["Player"])

In [6]:
predictors = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS','PER','TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM',
       'VORP']

#Predicting on 2024 players stats. Training model on data from previous years (1990 - 2023)
train = mvp_players
test = stats_2024

X_train = train[predictors]
y_train = train["Share"]

X_test = test[predictors]
# y_test = test["Share"]

model = XGBRegressor(learning_rate = 0.05, max_depth = 7, subsample = 0.5)

model.fit(X_train, y_train)
predicted_winners = model.predict(X_test)

predicted_winners = pd.DataFrame(predicted_winners, columns=["Predictions"], index = test.index)
combinations_XGB = pd.concat([test, predicted_winners], axis = 1)
combinations_XGB = combinations_XGB.sort_values("Predictions", ascending = False)
# combinations = combinations.sort_values("Share", ascending = False)
# combinations["Rank"] = list(range(1, combinations.shape[0] + 1))
# combinations = combinations.sort_values("Predictions", ascending = False)
# combinations["Predicted_Rank"] = list(range(1, combinations.shape[0] + 1))

In [7]:
combinations_XGB.head(10)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,VORP,Team,W,L,W/L%,GB,PS/G,PA/G,SRS,Predictions
68,Nikola Jokić,C,28.0,DEN,77.0,77.0,34.6,10.5,18.0,0.581,...,10.6,Denver Nuggets,0.0,0.0,0.000,0.0,0.0,0.0,0.00,0.849069
124,Shai Gilgeous-Alexander,PG,25.0,OKC,73.0,73.0,34.4,10.7,19.9,0.539,...,7.0,Oklahoma City Thunder,0.0,0.0,0.000,0.0,0.0,0.0,0.00,0.743793
31,Luka Dončić,PG,24.0,DAL,70.0,70.0,37.5,11.5,23.6,0.487,...,8.1,Dallas Mavericks,0.0,0.0,0.000,0.0,0.0,0.0,0.00,0.682518
7,Giannis Antetokounmpo,PF,29.0,MIL,73.0,73.0,35.2,11.5,18.8,0.611,...,7.2,Milwaukee Bucks,0.0,0.0,0.000,0.0,0.0,0.0,0.00,0.613293
244,Domantas Sabonis,C,27.0,SAC,79.0,79.0,35.7,7.7,13.0,0.593,...,6.1,Sacramento Kings,45.0,34.0,0.570,5.5,116.5,115.1,2.07,0.163223
167,Jayson Tatum,PF,25.0,BOS,73.0,73.0,35.8,9.1,19.4,0.472,...,4.7,Boston Celtics,0.0,0.0,0.000,0.0,0.0,0.0,0.00,0.080889
106,Tyrese Haliburton,PG,23.0,IND,67.0,66.0,32.3,7.3,15.3,0.476,...,4.9,Indiana Pacers,46.0,34.0,0.575,3.0,122.9,120.2,2.44,0.073800
186,Jalen Brunson,PG,27.0,NYK,74.0,74.0,35.4,10.1,21.2,0.478,...,5.1,New York Knicks,47.0,32.0,0.595,15.0,112.7,108.1,4.28,0.073137
334,Anthony Edwards,SG,22.0,MIN,77.0,76.0,35.2,9.2,20.0,0.462,...,3.8,Minnesota Timberwolves,0.0,0.0,0.000,0.0,0.0,0.0,0.00,0.071278
232,LeBron James,PF,39.0,LAL,69.0,69.0,35.1,9.6,17.8,0.538,...,5.1,Los Angeles Lakers,45.0,35.0,0.563,6.0,117.9,117.5,0.89,0.054277


In [8]:
combinations_XGB.head(5).to_csv("XGB_predictions.csv")

In [9]:
model = RandomForestRegressor(bootstrap=True, max_depth=40, max_features = 'sqrt', min_samples_leaf=2, min_samples_split=2, n_estimators = 200)

model.fit(X_train, y_train)
predicted_winners = model.predict(X_test)

predicted_winners = pd.DataFrame(predicted_winners, columns=["Predictions"], index = test.index)
combinations_RF = pd.concat([test, predicted_winners], axis = 1)
combinations_RF = combinations_RF.sort_values("Predictions", ascending = False)

In [10]:
combinations_RF.head(10)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,VORP,Team,W,L,W/L%,GB,PS/G,PA/G,SRS,Predictions
68,Nikola Jokić,C,28.0,DEN,77.0,77.0,34.6,10.5,18.0,0.581,...,10.6,Denver Nuggets,0.0,0.0,0.000,0.0,0.0,0.0,0.00,0.735313
124,Shai Gilgeous-Alexander,PG,25.0,OKC,73.0,73.0,34.4,10.7,19.9,0.539,...,7.0,Oklahoma City Thunder,0.0,0.0,0.000,0.0,0.0,0.0,0.00,0.502084
7,Giannis Antetokounmpo,PF,29.0,MIL,73.0,73.0,35.2,11.5,18.8,0.611,...,7.2,Milwaukee Bucks,0.0,0.0,0.000,0.0,0.0,0.0,0.00,0.484805
31,Luka Dončić,PG,24.0,DAL,70.0,70.0,37.5,11.5,23.6,0.487,...,8.1,Dallas Mavericks,0.0,0.0,0.000,0.0,0.0,0.0,0.00,0.440013
167,Jayson Tatum,PF,25.0,BOS,73.0,73.0,35.8,9.1,19.4,0.472,...,4.7,Boston Celtics,0.0,0.0,0.000,0.0,0.0,0.0,0.00,0.069428
186,Jalen Brunson,PG,27.0,NYK,74.0,74.0,35.4,10.1,21.2,0.478,...,5.1,New York Knicks,47.0,32.0,0.595,15.0,112.7,108.1,4.28,0.068558
220,Anthony Davis,C,30.0,LAL,74.0,74.0,35.5,9.3,16.8,0.554,...,4.7,Los Angeles Lakers,45.0,35.0,0.563,6.0,117.9,117.5,0.89,0.066022
244,Domantas Sabonis,C,27.0,SAC,79.0,79.0,35.7,7.7,13.0,0.593,...,6.1,Sacramento Kings,45.0,34.0,0.570,5.5,116.5,115.1,2.07,0.062397
334,Anthony Edwards,SG,22.0,MIN,77.0,76.0,35.2,9.2,20.0,0.462,...,3.8,Minnesota Timberwolves,0.0,0.0,0.000,0.0,0.0,0.0,0.00,0.045483
30,Kyrie Irving,SG,31.0,DAL,58.0,58.0,35.0,9.7,19.5,0.497,...,3.6,Dallas Mavericks,0.0,0.0,0.000,0.0,0.0,0.0,0.00,0.044326


In [11]:
combinations_RF.head(5).to_csv("RF_predictions.csv")